In [91]:
import pandas as pd
import awswrangler as awr
import openpyxl


In [92]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_baixadas.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_pagamentos = awr.athena.read_sql_query(query, database='silver')


In [93]:
df_pagamentos = df_pagamentos.drop_duplicates('ponteiro', keep='first')
df_pagamentos = df_pagamentos[df_pagamentos['data_baixa']>= pd.to_datetime('2025-01-31').date()]



In [94]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_inadimplentes.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_inadimplentes = awr.athena.read_sql_query(query, database='silver')
df_inadimplentes = df_inadimplentes.drop_duplicates('ponteiro', keep='first')

estabelecendo a base de vencidos

In [95]:
today = pd.to_datetime('today').date()
month = today.month
df_inadimplentes['data_vencimento'] = pd.to_datetime(df_inadimplentes['data_vencimento'])


base_vencidos = df_inadimplentes[df_inadimplentes['data_vencimento']>= pd.to_datetime('2025-01-31')]
#base_vencidos = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.month == month]


In [96]:
base_vencidos['situacao'] = pd.NA
base_vencidos['data_atualizacao'] = pd.NA
base_vencidos.sort_values(by='data_vencimento', ascending=False)
base_vencidos.head()


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_24780\1941425127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos['situacao'] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_24780\1941425127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos['data_atualizacao'] = pd.NA


,codigo_cadastro,associado,grupo,matricula,conjunto,aplicacao_financeira,data_emissao,data_vencimento,valor_titulo,codigo,...,numero_boleto,nosso_numero,status,id_set,parent,cooperativa,placa,chassi,situacao,data_atualizacao
0,8351,PIMENTEL COMERCIAL E TRANSPORTADORA LTDA,RECEITAS,13398,12523,Mensalidades/Benefícios,2024-12-02,2025-06-02,2872.97,1137,...,1167586,1167586-3,ATIVO,12523,13398,Stcoop,EJD1510,9BSR6X200L3964550,<NA>,<NA>
1,3566,EBC ENCOMENDAS LTDA,RECEITAS,19927,13260,Mensalidades/Benefícios,2025-01-23,2025-03-25,652.94,1361,...,1176179,1176179-2,ATIVO,13260,19927,Stcoop,SEE6H22,9VCVBYHVEPA005811,<NA>,<NA>
2,10507,DALLE TRANSPORTES LTDA.,RECEITAS,20253,11942,Mensalidades/Benefícios,2024-10-21,2025-06-02,1242.33,123,...,1158648,1158648-7,ATIVO,11942,20253,Stcoop,APW0B62,9BSP4X200C3803107,<NA>,<NA>
3,10760,TSUR TRANSPORTES LTDA,RECEITAS,20773,13483,Primeira Parcela,2025-02-06,2025-05-14,375.00,254,...,1179261,1179261-0,ATIVO,13483,20773,Stcoop,SJR3C44,9535E6TB7RR070140,<NA>,<NA>
4,10760,TSUR TRANSPORTES LTDA,RECEITAS,20773,12653,Mensalidades/Benefícios,2024-12-10,2025-04-16,3310.52,254,...,1187705,1187705-5,ATIVO,12653,20773,Stcoop,SVI7E22,9535E6TB2RR066173,<NA>,<NA>


acrescentando colunas importantes

In [97]:
df_pagamentos.loc[:, 'data_atualizacao'] = df_pagamentos['data_baixa']
df_pagamentos.loc[:, 'situacao']='PAGO'

base_vencidos.loc[:, 'data_atualizacao'] = base_vencidos['data_vencimento']
base_vencidos.loc[:, 'situacao']='INADIMPLENTE'

acrescentando colunas comuns e concatenando

In [98]:
colunas_comuns = [
    "matricula",
    "conjunto",
    "cooperativa",
    "ponteiro",
    "numero_boleto",
    "nosso_numero",
    "unidade",
    "associado",
    "data_vencimento",
    "data_baixa",
    "valor_titulo",
    "valor_baixa",
    "data_emissao",
    "dias_atrasado",
    "data_atualizacao",
    "situacao"
]

for col in colunas_comuns:
    if col not in df_pagamentos.columns:
        df_pagamentos.loc[:, col] = pd.NA
    if col not in base_vencidos.columns:
        base_vencidos.loc[:, col] = pd.NA

df_historico = pd.concat([df_pagamentos, base_vencidos], ignore_index=True)
df_historico = df_historico[colunas_comuns]


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_24780\1360921929.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos.loc[:, col] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_24780\1360921929.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos.loc[:, col] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_24780\1360921929.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version

In [99]:
df_historico['data_atualizacao'] = pd.to_datetime(df_historico['data_atualizacao'])
df_historico['data_atualizacao'] = df_historico['data_atualizacao'].dt.date


passando a base para o excel

In [102]:
save_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia"
base_vencidos.to_excel(save_path + r"\historico_faturas.xlsx", index=False, sheet_name='historico_faturas')


In [101]:
df_pagamentos.head()

,cooperativa,aplicacao,ponteiro,numero_boleto,nosso_numero,matricula,conjunto,unidade,associado,valor_baixa,data_baixa,data_vencimento,dias_atraso,data_atualizacao,situacao,valor_titulo,data_emissao,dias_atrasado
22,Stcoop,166,134418,1000950,24/000950-6,1528,8966,UNIDADE CASCAVEL,ALESSANDRO BETTIN CORONETI,1218.21,2025-04-07,2025-04-07,0,2025-04-07,PAGO,<NA>,<NA>,<NA>
90,Stcoop,166,110371,1104390,1104390-9,4475,7518,UNIDADE VILHENA,DALTO&DALTO LTDA-EPP,2445.32,2025-02-03,2025-02-03,0,2025-02-03,PAGO,<NA>,<NA>,<NA>
138,Stcoop,166,200040,1185519,1185519-6,6135,12863,UNIDADE CASCAVEL,CEZAR LUIZ CHIAPETTI E FILHO LTDA,663.29,2025-03-27,2025-03-27,0,2025-03-27,PAGO,<NA>,<NA>,<NA>
141,Stcoop,166,200609,1171369,1171369-7,328,12901,UNIDADE CASCAVEL,SANDRO DANIEL DALOSTO,1394.85,2025-05-05,2025-05-05,0,2025-05-05,PAGO,<NA>,<NA>,<NA>
200,Stcoop,166,145542,1007222,24/007222-4,5892,9544,UNIDADE CASCAVEL,MAURA BERTO RIBEIRO & CIA LTDA,1685.21,2025-04-07,2025-04-07,0,2025-04-07,PAGO,<NA>,<NA>,<NA>


AGORA É:

OK criar a base (já tá criada) 

OK gerar a base com vencidos este MÊS

ACIMA PODE GERAR DE NOVO SÓ RODANDO ISSO AMANHÃ, ATÉ O CÓDIGO DE ATT FICAR PRONTO OU SEGUIR ABAIXO


depois gerar o código de atualização nessa ideia:

todo dia vai (pegar o excel) vai (gerar a base de inadimplencia de hoje) e vai (concatenar as duas)

(gerar uma lista com os ponteiros vencidos neste dia do mês)

(gerar a lista de baixados neste mês) e (filtrar pela lista de ponteiros vencidos)

(concatenar esses pagamentos filtrados)

(atualizar o excel)